In [91]:
!pip install python-dotenv

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Found existing installation: pandas 2.2.3
Uninstalling pandas-2.2.3:
  Successfully uninstalled pandas-2.2.3


ERROR: Too many arguments


^C


In [9]:
import os
import requests
from dotenv import load_dotenv
import json
import pandas as pd

load_dotenv()

True

In [28]:
API_KEY = os.environ.get("API_KEY")

headers = {
        "accept": "application/json",
        "Authorization": f"Bearer {API_KEY}"
    }

def get_movie():
    url = "https://api.themoviedb.org/3/movie/12?language=en-US"
    response = requests.get(url, headers=headers)

    return response.text

def get_genres():
    url = "https://api.themoviedb.org/3/genre/movie/list?language=en"
    response = requests.get(url, headers=headers)

    return response.text

def popular_movies(page_num):
    url = f"https://api.themoviedb.org/3/movie/popular?language=en&page={page_num}"
    response = requests.get(url, headers=headers)

    return response.text

def popular_series():
    url = "https://api.themoviedb.org/3/tv/popular?language=en-US&page=1"
    response = requests.get(url, headers=headers)

    return response.text

In [15]:
json_string = get_genres()
data = json.loads(json_string)

genres_dict = {genre["id"]: genre["name"] for genre in data["genres"]}

genres_dict

{28: 'Action',
 12: 'Adventure',
 16: 'Animation',
 35: 'Comedy',
 80: 'Crime',
 99: 'Documentary',
 18: 'Drama',
 10751: 'Family',
 14: 'Fantasy',
 36: 'History',
 27: 'Horror',
 10402: 'Music',
 9648: 'Mystery',
 10749: 'Romance',
 878: 'Science Fiction',
 10770: 'TV Movie',
 53: 'Thriller',
 10752: 'War',
 37: 'Western'}

In [26]:
movies_csv_file = pd.DataFrame(columns=['Name','overview', 'genres', 'language', "popularity"]) 
movies_csv_file

,Name,overview,genres,language,popularity


In [27]:
pages = 500
for num in range(1, pages):  
    movies = popular_movies(page_num=num)
    movies_json = json.loads(movies)

    for movie in movies_json["results"]:
        genres = ", ".join([genres_dict.get(genre_id, "Unknown") for genre_id in movie["genre_ids"]])

        movie_data = pd.DataFrame([{
            'Name': movie["original_title"],
            'overview': movie["overview"],
            'genres': genres,
            'language': movie["original_language"],
            'popularity': movie["popularity"]
        }])

        movies_csv_file = pd.concat([movies_csv_file, movie_data], ignore_index=True)

movies_csv_file

C:\Users\Asus\AppData\Local\Temp/ipykernel_1412/1832397854.py:17: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  movies_csv_file = pd.concat([movies_csv_file, movie_data], ignore_index=True)


,Name,overview,genres,language,popularity
0,A Working Man,Levon Cade left behind a decorated military ca...,"Action, Crime, Thriller",en,581.2791
1,Havoc,When a drug heist swerves lethally out of cont...,"Action, Crime, Thriller",en,584.7785
2,A Minecraft Movie,Four misfits find themselves struggling with o...,"Family, Comedy, Adventure, Fantasy",en,408.6823
3,Van Gogh by Vincent,"In a career that lasted only ten years, Vincen...",Documentary,en,389.5467
4,新幹線大爆破,When panic erupts on a Tokyo-bound bullet trai...,"Action, Thriller, Crime, Drama",ja,326.4027
5,In the Lost Lands,A queen sends the powerful and feared sorceres...,"Action, Fantasy, Adventure",en,288.7711
6,The Siege,International assassin Walker is compromised d...,"Action, Thriller",en,248.6815
7,ज्वेल थीफ: द हीस्ट बिगिन्स,"In this high-octane battle of wits and wills, ...","Action, Thriller",hi,271.0703
8,Captain America: Brave New World,After meeting with newly elected U.S. Presiden...,"Action, Thriller, Science Fiction",en,247.2469
9,Desert Dawn,A newly appointed small-town sheriff and his b...,"Action, Crime, Mystery, Thriller",en,248.4328


In [ ]:
movies_csv_file.to_csv('movies.csv', index=False)